## Convert Data

Useful notebook to visualize and convert data to npz format for general testing

Need to figure out how this was simulated in the CEVAE paper

In [1]:
import numpy as np
import pandas as pd
from scipy.special import expit
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm

# Random generators initialization
import random
seed=42
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
print('[INFO] Random generators were initialized')

[INFO] Random generators were initialized


In [2]:
df = pd.read_csv('./Raw_Data/TWINS/twin_pairs_X_3years_samesex.csv')
ys = pd.read_csv('./Raw_Data/TWINS/twin_pairs_Y_3years_samesex.csv').drop(['Unnamed: 0'], axis=1)
weight = pd.read_csv('./Raw_Data/TWINS/twin_pairs_T_3years_samesex.csv').drop(['Unnamed: 0'], axis=1)


df = df[ (weight['dbirwt_0'] >= 2000) & (weight['dbirwt_1'] >= 2000) ]
ys = ys[ (weight['dbirwt_0'] >= 2000) & (weight['dbirwt_1'] >= 2000) ]


In [3]:
desc = open('./Raw_Data/TWINS/covar_desc.txt', 'r').read()
desc = eval(desc)

types = open('./Raw_Data/TWINS/covar_type.txt', 'r').read()
types = eval(types)

types['gestat10'] = 'ord'

In [4]:
df['bord'] = (df['bord_0'] < df['bord_1']).astype(int)
to_remove = ['Unnamed: 0', 'Unnamed: 0.1', 'infant_id_0', 'infant_id_1',
             'brstate', 'stoccfipb', 'mplbir', 'bord_0', 'bord_1']
df = df.drop(to_remove, axis=1)

for var in to_remove + ['gestat10']:
    if var in types:
        types.pop(var)

In [5]:
group_vars = {}
for key, value in types.items():
    group_vars[value] = group_vars.get(value, []) + [key]

In [6]:
ys = ys.loc[ df.dropna().index ]
df = df.loc[ df.dropna().index ]

# missing = df.isna().mean(axis=0) > 0.2

# max_values = (df.max(axis=0) + 1)[missing]
# # print (max_values.shape)

# mode_values = df.mode(axis=0).iloc[0][np.logical_not(missing)]
# # print (mode_values.shape)


# new_category = missing.index[missing]
# mode_category = missing.index[np.logical_not(missing)]

# print ("These columns are imputed using max_val + 1")
# print (new_category)

# print ("These columns are imputed using mode")
# print (mode_category)

# df[new_category] = df[new_category].fillna(max_values, axis=0)
# df[mode_category] = df[mode_category].fillna(mode_values, axis=0)

In [7]:
df = pd.get_dummies(df, columns=group_vars['cat'])
print (df.shape)
print ("This is not the same as CEVAE but the closest we could get to the author's description")

(23905, 146)
This is not the same as CEVAE but the closest we could get to the author's description


In [8]:
z = df['gestat10'].values.reshape(-1,1)
x = df.drop(['gestat10'], axis=1).values

In [9]:
n = 5
    
w0 = 0.1  * np.random.randn(x.shape[1], n) 
wh = 5 + 0.1 * np.random.randn(1, n)
probs = expit(x @ w0 + (z / 10 - 0.1) @ wh)
t = np.random.binomial(1, probs)

ys = ys.values

In [12]:
noises = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
problem_id = 0
for noise in noises:
    print (f"Noise: {100*noise:.0f}%")
    prox = pd.get_dummies(df['gestat10']).values[:, :, np.newaxis]
    prox = np.repeat(prox, 3, 1)
    prox = np.repeat(prox, n, 2).astype(bool)
    flip = (np.random.uniform(size=prox.shape) > (1-noise))
    proxies = np.logical_xor(prox, flip).astype(int)

    x_repeat = np.repeat(x[:, :, np.newaxis], n, 2)
    features = np.concatenate([x_repeat, proxies], axis=1)
    
    path = "../Data/Twins"
    
    if not os.path.exists(path):
        os.makedirs(path)
    count = features.shape[0]
    size = int(0.75 * count)

    for i in tqdm(range(n)):
        temp_x = features[:,:,i]
        temp_t = t[:,i].astype(int)
        temp_yf = ys[np.arange(ys.shape[0]), temp_t]
        temp_ycf = ys[np.arange(ys.shape[0]), 1-temp_t]
        temp_mu0 = ys[:, 0]
        temp_mu1 = ys[:, 1]
        
        x_train, x_test, t_train, t_test, yf_train, yf_test, \
           ycf_train, ycf_test, mu0_train, mu0_test, mu1_train, mu1_test, \
            = train_test_split(temp_x, temp_t, temp_yf, temp_ycf, temp_mu0, temp_mu1, train_size=0.8, random_state=42, shuffle=True)

        np.savez(path + f'/train{problem_id}.npz', x=x_train, t=t_train, yf=yf_train, ycf=ycf_train, mu1=mu1_train, mu0=mu0_train)
        np.savez(path + f'/test{problem_id}.npz', x=x_test, t=t_test, yf=yf_test, ycf=ycf_test, mu1=mu1_test, mu0=mu0_test)

        # Update counter
        problem_id += 1

Noise: 0%


100%|██████████| 5/5 [00:00<00:00,  8.14it/s]


Noise: 10%


100%|██████████| 5/5 [00:00<00:00,  8.81it/s]


Noise: 20%


...

100%|██████████| 5/5 [00:00<00:00,  7.69it/s]


Noise: 40%


100%|██████████| 5/5 [00:00<00:00,  7.38it/s]


Noise: 50%


100%|██████████| 5/5 [00:00<00:00,  7.81it/s]
